# A quick overview of Cython

## Take one: Cheap speedups by Cythonizing
We can define a native python Fibonacci function;

In [25]:
def pyfib(n):
    if n <= 1:
        return 1
    return pyfib(n-1)+pyfib(n-2)

n = 30

In [9]:
%timeit pyfib(n)

806 ms ± 38.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Let's say we've tried this, and we figure it's running a bit too slowly. That's not unreasonable, maybe there's a lot of overhead from Python recursive calls.

Jupyter has an easy way to incorporate on-the-fly Cythonizing of functions. We can load the Cython extension by,

In [5]:
%load_ext cython

The cython extension is already loaded. To reload it, use:
  %reload_ext cython


And then with the following magic, we can start writing Cython code that will be automatically compiled and reimported at our beck and call

In [6]:
%%cython

# literally a copy and paste
def cy_pyfib(n):
    if n <= 1:
        return 1
    return cy_pyfib(n-1)+cy_pyfib(n-2)

# make some small C-ifications
cpdef int cyjufib(int n):
    if n <= 1:
        return 1
    return cyjufib(n-2)+cyjufib(n-1)

Now let's run some profiles on our code.

In [8]:
n = 30

In [10]:
%timeit cy_pyfib(n)

531 ms ± 29.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
%timeit cyjufib(n)

8.25 ms ± 47.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


We can see that the Cythonized and C code ended up being 100x faster!

If we already had the C code implemented in a C file, we can use Cython to *talk to C* and call C functions from Python

In [27]:
%%cython -Smy_library.c -I.

cdef extern from "my_library.h":
    int fib(int n)

def c_fib(int n):
    return fib(n)

In [12]:
%timeit c_fib(n)

8.6 ms ± 440 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Of course, one of the main problems with our function is that it is slow because *the implementation is slow*. 

## Take two: Speedup from better implementation
If we unroll the recursion, we might get some speed increases. In pure Python;

In [13]:
def pyfib_unroll(n):
    if n <= 1:
        return 1
    f_n2,f_n1 = 1,1
    for i in range(n-2):
        f_n2,f_n1 = f_n1,f_n1+f_n2
    return f_n1+f_n2

In [15]:
%timeit pyfib_unroll(n)

5.24 µs ± 64.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


If you're counting, that's on the order of a \\( 10^{6} \\) speedup over the original Python! Orders faster than the Cythonizing. So when you are trying to optimize your code, keep in mind that much of your issue may be an *inefficient implementation*, rather than it being Python's fault.

Now, the Fibonacci code is pretty simple, and it's not surprising that Cythonizing does improve the runtime a little,

In [14]:
%%cython

# literally a copy-paste
def cy_pyfib_unroll(n):
    if n <= 1:
        return 1
    f_n2,f_n1 = 1,1
    for i in range(n-2):
        f_n2,f_n1 = f_n1,f_n1+f_n2
    return f_n1+f_n2

# add some c-ification
cpdef int cyjufib_unroll(int n):
    cdef int f_n2,f_n1
    cdef int i
    if n <= 1:
        return 1
    f_n2,f_n1 = 1,1
    for i in range(n-2):
        f_n2,f_n1 = f_n1,f_n1+f_n2
    return f_n1+f_n2

In [16]:
%timeit cy_pyfib_unroll(n)

3.49 µs ± 81.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [17]:
%timeit cyjufib_unroll(n)

237 ns ± 3.13 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


We also have an unrolled function in C, too, so let's wrap it:

In [24]:
%%cython -Smy_library.c

cdef extern from "my_library.h":
    int fib_unroll(int n)

def c_fib_unroll(int n):
    return fib_unroll(n)

In [18]:
%timeit c_fib_unroll(n)

306 ns ± 155 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


Here we can see that Cythonizing now only adds an extra 20 times speedup, give or take. It's certainly an improvement, but even the new Python implementation is faster, and may already be fast enough for what you need. "Premature optimization is the root of all evil," so remember that this small extra speedup of Cython comes with the drawbacks of, for instance, time spent optimizing and losing pdb debugging functionality.

Of course, if we are willing to use a little extra memory, it's easy to create a Python Fibonacci function that's as quick as the C functions using result caching, which is super easy with decorators:

In [19]:
from functools import lru_cache

@lru_cache(maxsize=None)
def py_fib_cache(n):
    if n <= 1:
        return 1
    return py_fib_cache(n-2)+py_fib_cache(n-1)

In [20]:
%timeit py_fib_cache(n)

408 ns ± 130 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)



Let's check to make sure that all of these different functions are actually giving the right answers, of course...

In [21]:
pyfib(n), cy_pyfib(n), cyjufib(n), c_fib(n), pyfib_unroll(n), cy_pyfib_unroll(n), cyjufib_unroll(n), c_fib_unroll(n), py_fib_cache(n)

(1346269,
 1346269,
 1346269,
 1346269,
 1346269,
 1346269,
 1346269,
 1346269,
 1346269)

As a last comment, Cython magic makes it really easy to use cython's annotator:

In [22]:
%%cython -a

def cy_pyfib(n):
    if n <= 1:
        return 1
    return cy_pyfib(n-1)+cy_pyfib(n-2)

def cy_pyfib_unroll(n):
    if n <= 1:
        return 1
    f_n2,f_n1 = 1,1
    for i in range(n-2):
        f_n2,f_n1 = f_n1,f_n1+f_n2
    return f_n1+f_n2

cpdef int cyjufib(int n):
    if n <= 1:
        return 1
    return cyjufib(n-2)+cyjufib(n-1)

cpdef int cyjufib_unroll(int n):
    cdef int f_n2,f_n1
    cdef int i
    if n <= 1:
        return 1
    f_n2,f_n1 = 1,1
    for i in range(n-2):
        f_n2,f_n1 = f_n1,f_n1+f_n2
    return f_n1+f_n2

The more yellow a line, the more "Python interaction" the line has when it's compiled with Cython. Heuristically, the more yellow lines can be optimized a bit more by being more explicit.

Clicking any line will show what C code Cython has compiled that line to.

## Summary

1. We've covered two things that Cython lets you do:
    1.  Cython (sometimes) can help your code run faster by reducing Python's overhead
    2.  Cython gives you an easy Python-like way to write interfaces to C code, for experimentation
        in Python
2. Cython can give you some small boosts with little extra work, but by
   being more explicit, you can squeeze out even more performance!
3. Often the easiest way to speed up your code is by finding a better implementation